# Modeling Example: The Blog Application

Modeling an application is an iterative process.
It involves the use of several fundamental principles and patterns.
This document walks through that process using the familiar blog as an example.

In [1]:
// Reference the Jinaga NuGet packages
#r "nuget: Jinaga, 0.11.18"
#r "nuget: Jinaga.Graphviz, 0.11.18"
#r "nuget: Jinaga.UnitTest, 0.11.18"

Installed Packages Jinaga, 0.11.18 Jinaga.Graphviz, 0.11.18 Jinaga.UnitTest, 0.11.18

In [2]:
using Jinaga;
using Jinaga.Graphviz;
using Jinaga.UnitTest;

// Create a Jinaga client for unit testing
var jinagaClient = JinagaTest.Create(opt =>
{
    // Simulate a logged in user
    opt.User = new User("--- FAKE USER ---");
});

## Declaring Facts

Express a Jinaga model as a set of C# records representing historical facts.
Decorate each record with a `FactType` attribute.
The fields of the records are predecessors -- facts that came before -- and fields.

In the Blog model, a site is an entity created by a specific user.
We use the date and time to distinguish it from other sites that the user created.
The assumption is that a user will not create multiple sites within the same millisecond.

Notice that the site does not contain any other properties, such as a title or a domain name.
Those properties are mutable, and therefore not part of the initial fact.

In [3]:
[FactType("Blog.Site")]
public record Site(User creator, DateTime createdAt) { }

The `User` type is provided by the Jinaga library.
It represents someone who can log into the app.
A model almost always starts with a fact owned by a user.
This gives us a place to start our authorization rules.

In [4]:
// Call RenderTypes without a semicolon to display the graph
Renderer.RenderTypes(typeof(User), typeof(Site))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

## Log In

The user is the starting point of the model.
You'll need to call `Login` to get the logged in user.

In [5]:
// Login returns the user fact and profile information
var (user, profile) = await jinagaClient.Login();

// Call RenderFacts on the Jinaga client to display the facts
jinagaClient.RenderFacts(user)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER ---

## Save Facts

Save an instance of a fact to the Jinaga client whenever the user takes an action.
Pass a record to the `Fact` method.
This will save the fact in the local store and share it with the server.

The `Fact` method returns the record that was just saved.
Hold on to this copy of the record for use in other methods.

In [6]:
var site = await jinagaClient.Fact(new Site(user, DateTime.UtcNow));

jinagaClient.RenderFacts(site)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator

## Query for Successors

Given one fact, you can find its successors using LINQ.
For example, you can find all sites for a given user.
Use `Given<T>.Match()` to define a specification.

In [7]:
// The parameters to the lambda are the given fact (in this case, the user) and
// the fact repository. Use the OfType<T> method to get facts from the repository.
var sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    select site);

// Query for facts matching the specification.
var sites = await jinagaClient.Query(sitesByUser, user);

jinagaClient.RenderFacts(sites)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator

## Deleting Facts

Facts are not truly deleted.
You indicate that they should be deleted by introducing a new fact.
There are mechanisms for taking them out of storage, but we'll cover those later.

To indicate that a site should be deleted, define a fact that refers to the site as a predecessor.
It also should have a timestamp so that we can differentiate one deletion from another.

In [8]:
[FactType("Blog.Site.Deleted")]
public record SiteDeleted(Site site, DateTime deletedAt) { }

Renderer.RenderTypes(typeof(SiteDeleted))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site.Deleted -->
 
 Blog.Site.Deleted 
 
 Blog.Site.Deleted 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site.Deleted->Blog.Site -->
 
 Blog.Site.Deleted->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

To indicate that a site should be deleted, create an instance of that fact.

In [9]:
var siteDeleted = await jinagaClient.Fact(new SiteDeleted(site, DateTime.UtcNow));

jinagaClient.RenderFacts(siteDeleted)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA== -->
 
 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA== 
 
 Blog.Site.Deleted 
 
 deletedAt 
 
 2024-05-18T17:54:05.... 
 
 
<!-- 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site

If you query the specification now, you will still see the site.

In [10]:
sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

1

That's because we need to change the specification to exclude deleted sites.
Add a clause that filters out sites that have a site deleted successor.

In [11]:
sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    // Include only the sites that have not been deleted
    where !facts.Any<SiteDeleted>(sd => sd.site == site)
    select site);

sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

0

## Restoring Facts

Users sometimes make mistakes.
They should be able to undo a deletion.
To allow this, create a new fact type that represents the restoration.

To restore a site, define a fact type that refers to the deletion of that site.
It needs no additional parameters.

In [12]:
[FactType("Blog.Site.Restored")]
public record SiteRestored(SiteDeleted deleted) { }

Renderer.RenderTypes(typeof(SiteRestored))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site.Restored -->
 
 Blog.Site.Restored 
 
 Blog.Site.Restored 
 
<!-- Blog.Site.Deleted -->
 
 Blog.Site.Deleted 
 
 Blog.Site.Deleted 
 
<!-- Blog.Site.Restored->Blog.Site.Deleted -->
 
 Blog.Site.Restored->Blog.Site.Deleted 
 
 
 deleted 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site.Deleted->Blog.Site -->
 
 Blog.Site.Deleted->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

Create an instance of this fact to indicate that the deletion should no longer take effect.

In [13]:
var siteRestored = await jinagaClient.Fact(new SiteRestored(siteDeleted));

jinagaClient.RenderFacts(siteRestored)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA== -->
 
 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA== 
 
 Blog.Site.Deleted 
 
 deletedAt 
 
 2024-05-18T17:54:05.... 
 
<!-- 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- sWnBxx6intvnMSvG6l0p9EFHnwKPBahcLgx4XNQMy5y4+QE3BlikuD6enPtJlj4FCAas/QCDrAKzcZgva3xEDQ== -->
 
 sWnBxx6intvnMSvG6l0p9EFHnwKPBahcLgx4XNQMy5y4+QE3BlikuD6enPtJlj4FCAas/QCDrAKzcZgva3xEDQ== 
 
 Blog.Site.Restored 
 
 
<!-- sWnBxx6intvnMSvG6l0p9EFHnwKPBahcLgx4XNQMy5y4+QE3BlikuD6enPtJlj4FCAas/QCDrAKzcZgva3xEDQ==->2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA== -->
 
 sWnBxx6intvnMSvG6l0p9EFHnwKPBahcLgx4XNQMy5y4+QE3BlikuD6enPtJlj4FCAas/QCDrAKzcZgva3xEDQ==->2YV8flHkVNvd1N9cX77/1gayAd4ZgMXpttzE/IFlkoKwfrIgpO070IEDXLe0XOSNd0pQkB/Lmj4hGwWBsRrSvA== 
 
 
 deleted

As you might imagine, the specification doesn't honor this new fact yet.

In [14]:
sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

0

But if we filter the deletions to include only those that don't have a successor restore fact, then we get the desired behavior.

In [15]:
sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    where !facts.Any<SiteDeleted>(
        sd => sd.site == site &&
        // Honor only the site deletion that have not been restored
        !facts.Any<SiteRestored>(sr => sr.deleted == sd))
    select site);

sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

1

## Mutable Properties

Facts are immutable.
But sometimes we want to record the values of properties that can change over time.
To do so, define a new fact type representing a change to that value.

The name of a site should be allowed to change.
That is why we didn't include it as a field of the `Site` fact.
To model this, define a `SiteName` fact that refers to the site and stores the new value.
It should also refer to past `SiteName` facts that it replaces.

In [16]:
[FactType("Blog.Site.Name")]
public record SiteName(Site site, string value, SiteName[] prior) { }

Renderer.RenderTypes(typeof(SiteName))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site.Name -->
 
 Blog.Site.Name 
 
 Blog.Site.Name 
 
<!-- Blog.Site.Name->Blog.Site.Name -->
 
 Blog.Site.Name->Blog.Site.Name 
 
 
 prior 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site.Name->Blog.Site -->
 
 Blog.Site.Name->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

When we create the first instance of the site name fact, we have no prior names to replace.
So we pass in an empty array.

In [17]:
var siteName0 = await jinagaClient.Fact(new SiteName(site, "My Site", []));

jinagaClient.RenderFacts(siteName0)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== 
 
 Blog.Site.Name 
 
 value 
 
 My Site 
 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site

If the user changes the name of the site, then we record that with a new fact that replaces the first one.

In [18]:
var siteName1 = await jinagaClient.Fact(new SiteName(site, "My Blog", [siteName0]));

jinagaClient.RenderFacts(siteName1)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== 
 
 Blog.Site.Name 
 
 value 
 
 My Site 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== 
 
 Blog.Site.Name 
 
 value 
 
 My Blog 
 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== 
 
 
 prior

If they change it again, we only include the most recent value that we are replacing.
There is no need to list all of the past values, since some of them have already been replaced.
This forms a chain of values that the property took on over time.

In [19]:
var siteName2 = await jinagaClient.Fact(new SiteName(site, "My Journal", [siteName1]));

jinagaClient.RenderFacts(siteName2)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== 
 
 Blog.Site.Name 
 
 value 
 
 My Site 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== 
 
 Blog.Site.Name 
 
 value 
 
 My Blog 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== 
 
 
 prior 
 
<!-- WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA== -->
 
 WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA== 
 
 Blog.Site.Name 
 
 value 
 
 My Journal 
 
 
<!-- WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA==->9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== -->
 
 WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA==->9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== 
 
 
 prior

To find the current name of a site, we look for site names that have not been replaced.
Let's take this in two parts.

First, look for all of the names of a site.
This will include all past values.

In [20]:
var namesOfSite = Given<Site>.Match((site, facts) =>
    from name in facts.OfType<SiteName>()
    where name.site == site
    select name);

var names = await jinagaClient.Query(namesOfSite, site);

names.Count()

3

Second, let's filter this history.
We only want the names for which there is no next value.

In [21]:
namesOfSite = Given<Site>.Match((site, facts) =>
    from name in facts.OfType<SiteName>()
    where name.site == site
    // Filter out names for which a next name exists
    where !facts.Any<SiteName>(next => next.prior.Contains(name))
    select name);

names = await jinagaClient.Query(namesOfSite, site);

names.Count()

1

In [22]:
names.Single().value

My Journal

## Projections

To populate a user interface, a specification will need to add details to its results.
To accomplish this, create an object in the `select` clause.
The object can include properties that are computed from child specifications.

For the blog application, we'll want to display a list of sites.
The user will need to see the name of each site in the list.
Let's modify the `sitesByUser` specification to include the names of each site.

In [23]:
var sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    where !facts.Any<SiteDeleted>(
        sd => sd.site == site &&
        !facts.Any<SiteRestored>(sr => sr.deleted == sd))
    // Select an anonymous object with information about the site
    select new
    {
        // Include the site fact
        Site = site,
        // Get the list of names for the site
        Names =
            from name in facts.OfType<SiteName>()
            where name.site == site
            where !facts.Any<SiteName>(next => next.prior.Contains(name))
            // Pick the value, not the fact
            select name.value
    });

var sites = await jinagaClient.Query(sitesByUser, user);

sites

index value 0 { Site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, Names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } Site Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM } creator Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:05Z Names [ My Journal ] (values) [ My Journal ]

With this structure, the user interface can generate a list of sites.
The `Names` property is going to be a collection of names, not a single value.
The user interface might use `FirstOrDefault` to turn it into a single value.

In [24]:
var sitesViewModel = sites.Select(s => new
{
    Name = s.Names.FirstOrDefault() ?? "New site"
});

sitesViewModel

index value 0 { Name = My Journal } Name My Journal

## Additional Mutable Properties

To add mutable properties to the model, keep defining new facts.
Add them to the projection to get a complete picture of your entities.

In [25]:
[FactType("Blog.Site.Domain")]
public record SiteDomain(Site site, string value, SiteDomain[] prior) { }

Renderer.RenderTypes(typeof(SiteDomain), typeof(SiteName))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site.Domain -->
 
 Blog.Site.Domain 
 
 Blog.Site.Domain 
 
<!-- Blog.Site.Domain->Blog.Site.Domain -->
 
 Blog.Site.Domain->Blog.Site.Domain 
 
 
 prior 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site.Domain->Blog.Site -->
 
 Blog.Site.Domain->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator 
 
<!-- Blog.Site.Name -->
 
 Blog.Site.Name 
 
 Blog.Site.Name 
 
<!-- Blog.Site.Name->Blog.Site -->
 
 Blog.Site.Name->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Name->Blog.Site.Name -->
 
 Blog.Site.Name->Blog.Site.Name 
 
 
 prior

In [26]:
var siteDomain0 = await jinagaClient.Fact(new SiteDomain(site, "example.com", []));

jinagaClient.RenderFacts(siteDomain0, siteName2)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- eeJZNi8jUtVkOHs2ZOx51b9Ey3365HTVGDcP756b1JRkFxd8VdTZ4Vuy/TC6r61iylAB3kfSU+qQAYbGnZMAGg== -->
 
 eeJZNi8jUtVkOHs2ZOx51b9Ey3365HTVGDcP756b1JRkFxd8VdTZ4Vuy/TC6r61iylAB3kfSU+qQAYbGnZMAGg== 
 
 Blog.Site.Domain 
 
 value 
 
 example.com 
 
 
<!-- eeJZNi8jUtVkOHs2ZOx51b9Ey3365HTVGDcP756b1JRkFxd8VdTZ4Vuy/TC6r61iylAB3kfSU+qQAYbGnZMAGg==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 eeJZNi8jUtVkOHs2ZOx51b9Ey3365HTVGDcP756b1JRkFxd8VdTZ4Vuy/TC6r61iylAB3kfSU+qQAYbGnZMAGg==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== 
 
 Blog.Site.Name 
 
 value 
 
 My Site 
 
<!-- 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A== 
 
 Blog.Site.Name 
 
 value 
 
 My Blog 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== -->
 
 9dC35UJCGS0NM9MGKxWXUYboZoz5VDtQwD1Cgu73+YH6yqh9IehqLwwLpkUDllpUeYTCQOFrs8wjl4LXYNy18A==->0XrupJGM1PkDI2fC5bi+svuXh3THNZZlwAymjsmsq8HTRrDF+3tv/F3WJ2dWXUsBSbbJeH5AY8RBrFaaiGGdEw== 
 
 
 prior 
 
<!-- WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA== -->
 
 WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA== 
 
 Blog.Site.Name 
 
 value 
 
 My Journal 
 
 
<!-- WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 WXPpGmREOVopsJKw5KziUFyBOUz7pU5AnWswU6qmsA0/lhXFmxQYt/MeDkdktPfWKqOQRPdBkg6HB7Xnyv1MvA==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- WX

In [27]:
var sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    where !facts.Any<SiteDeleted>(
        sd => sd.site == site &&
        !facts.Any<SiteRestored>(sr => sr.deleted == sd))
    select new
    {
        Site = site,
        Names =
            from name in facts.OfType<SiteName>()
            where name.site == site
            where !facts.Any<SiteName>(next => next.prior.Contains(name))
            select name.value,
        Domains =
            from domain in facts.OfType<SiteDomain>()
            where domain.site == site
            where !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
            select domain.value
    });

var sites = await jinagaClient.Query(sitesByUser, user);

var sitesViewModel = sites.Select(s => new
{
    Name = s.Names.FirstOrDefault() ?? "New site",
    Domain = s.Domains.FirstOrDefault() ?? ""
});

sitesViewModel

index value 0 { Name = My Journal, Domain = example.com } Name My Journal Domain example.com

## Child Objects

Most applications have a hierarchical structure.
The top-level objects contain child objects.
To model this, create facts that refer back to their parents.

A user can create posts within a site.
That user might be the creator of the blog, or it might be a different user.
A `Post` fact refers to the `Site` parent, the author `User`, and also includes a timestamp to differentiate it from other posts.

In [28]:
[FactType("Blog.Post")]
public record Post(Site site, User author, DateTime createdAt) { }

Renderer.RenderTypes(typeof(Post))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Post -->
 
 Blog.Post 
 
 Blog.Post 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Post->Blog.Site -->
 
 Blog.Post->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Post->Jinaga.User -->
 
 Blog.Post->Jinaga.User 
 
 
 author 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

In [29]:
var post0 = await jinagaClient.Fact(new Post(site, user, DateTime.UtcNow));
var post1 = await jinagaClient.Fact(new Post(site, user, DateTime.UtcNow));

jinagaClient.RenderFacts(post0, post1)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== -->
 
 NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== 
 
 Blog.Post 
 
 createdAt 
 
 2024-05-18T17:54:09.... 
 
 
<!-- NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 author 
 
<!-- NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 Blog.Post 
 
 createdAt 
 
 2024-05-18T17:54:09.... 
 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 author 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site

When using the application, you will first select a site from the list.
Then you will navigate to a page where you see the posts.
The specification should therefore start from the site.

In [30]:
var postsInSite = Given<Site>.Match((site, facts) =>
    from post in facts.OfType<Post>()
    where post.site == site
    select post);

var posts = await jinagaClient.Query(postsInSite, site);

posts.Count()

2

Continue the pattern to define deletion, restoration, and mutable properties for the child objects.

In [31]:
[FactType("Blog.Post.Title")]
public record PostTitle(Post post, string value, PostTitle[] prior) { }

[FactType("Blog.Post.Content")]
public record PostContent(Post post, string markdown, PostContent[] prior) { }

[FactType("Blog.Post.Deleted")]
public record PostDeleted(Post post, DateTime deletedAt) { }

[FactType("Blog.Post.Restored")]
public record PostRestored(PostDeleted deleted) { }

Renderer.RenderTypes(typeof(PostTitle), typeof(PostContent), typeof(PostDeleted), typeof(PostRestored))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Post.Title -->
 
 Blog.Post.Title 
 
 Blog.Post.Title 
 
<!-- Blog.Post.Title->Blog.Post.Title -->
 
 Blog.Post.Title->Blog.Post.Title 
 
 
 prior 
 
<!-- Blog.Post -->
 
 Blog.Post 
 
 Blog.Post 
 
<!-- Blog.Post.Title->Blog.Post -->
 
 Blog.Post.Title->Blog.Post 
 
 
 post 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Post->Blog.Site -->
 
 Blog.Post->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Post->Jinaga.User -->
 
 Blog.Post->Jinaga.User 
 
 
 author 
 
<!-- Blog.Post.Content -->
 
 Blog.Post.Content 
 
 Blog.Post.Content 
 
<!-- Blog.Post.Content->Blog.Post -->
 
 Blog.Post.Content->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Content->Blog.Post.Content -->
 
 Blog.Post.Content->Blog.Post.Content 
 
 
 prior 
 
<!-- Blog.Post.Deleted -->
 
 Blog.Post.Deleted 
 
 Blog.Post.Deleted 
 
<!-- Blog.Post.Deleted->Blog.Post -->
 
 Blog.Post.Deleted->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Restored -->
 
 Blog.Post.Restored 
 
 Blog.Post.Restored 
 
<!-- Blog.Post.Restored->Blog.Post.Deleted -->
 
 Blog.Post.Restored->Blog.Post.Deleted 
 
 
 deleted 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

In [32]:
var postTitle0 = await jinagaClient.Fact(new PostTitle(post0, "Welcome to My Blog", []));
var postContent0 = await jinagaClient.Fact(new PostContent(post0, "This is my first post.", []));

var postTitle1 = await jinagaClient.Fact(new PostTitle(post1, "Interesting Facts", []));
var postContent1 = await jinagaClient.Fact(new PostContent(post1, "Let me tell you about this thing that I love.", []));

var post0Deleted = await jinagaClient.Fact(new PostDeleted(post0, DateTime.UtcNow));

jinagaClient.RenderFacts(postTitle0, postContent0, postTitle1, postContent1, post0Deleted)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== -->
 
 NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== 
 
 Blog.Post 
 
 createdAt 
 
 2024-05-18T17:54:09.... 
 
<!-- NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 author 
 
<!-- NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- qyyjMBQ9WrEeCJO7V/XPiLY+42Wa9MR6CrmN99h9cHaArszo3i94frCHQKtjgGmz1AT3DmuABrvVUIFWWZyqvw== -->
 
 qyyjMBQ9WrEeCJO7V/XPiLY+42Wa9MR6CrmN99h9cHaArszo3i94frCHQKtjgGmz1AT3DmuABrvVUIFWWZyqvw== 
 
 Blog.Post.Title 
 
 value 
 
 Welcome to My Blog 
 
 
<!-- qyyjMBQ9WrEeCJO7V/XPiLY+42Wa9MR6CrmN99h9cHaArszo3i94frCHQKtjgGmz1AT3DmuABrvVUIFWWZyqvw==->NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== -->
 
 qyyjMBQ9WrEeCJO7V/XPiLY+42Wa9MR6CrmN99h9cHaArszo3i94frCHQKtjgGmz1AT3DmuABrvVUIFWWZyqvw==->NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== 
 
 
 post 
 
<!-- Tgzp2TZY6Klk5GFrOCXUjUW6RFS6kcHUo94PUWLAvIrDMZpp0A0TDmFKmJ1N85uP6Ql2WkjuJ6NQ+rSVEh7CqA== -->
 
 Tgzp2TZY6Klk5GFrOCXUjUW6RFS6kcHUo94PUWLAvIrDMZpp0A0TDmFKmJ1N85uP6Ql2WkjuJ6NQ+rSVEh7CqA== 
 
 Blog.Post.Content 
 
 markdown 
 
 This is my first pos... 
 
 
<!-- Tgzp2TZY6Klk5GFrOCXUjUW6RFS6kcHUo94PUWLAvIrDMZpp0A0TDmFKmJ1N85uP6Ql2WkjuJ6NQ+rSVEh7CqA==->NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== -->
 
 Tgzp2TZY6Klk5GFrOCXUjUW6RFS6kcHUo94PUWLAvIrDMZpp0A0TDmFKmJ1N85uP6Ql2WkjuJ6NQ+rSVEh7CqA==->NlHSw2QTuLZc5tlNFCqKlkH4DoYUW3GwL2JYDlOoMnnG2ECGhl4a20V822tmysx2aJ7wCjk6tRxm/MHEnFYWWg== 
 
 
 post 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 Blog.Post 
 
 createdAt 
 
 2024-05-18T17:54:09.... 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBS

Extend the specification to filter out deleted posts and project properties such as title.

In [33]:
var postsInSite = Given<Site>.Match((site, facts) =>
    from post in facts.OfType<Post>()
    where post.site == site
    where !facts.Any<PostDeleted>(pd => pd.post == post &&
        !facts.Any<PostRestored>(pr => pr.deleted == pd))
    select new
    {
        Post = post,
        Title =
            from title in facts.OfType<PostTitle>()
            where title.post == post
            where !facts.Any<PostTitle>(next => next.prior.Contains(title))
            select title.value,
        Content =
            from content in facts.OfType<PostContent>()
            where content.post == post
            where !facts.Any<PostContent>(next => next.prior.Contains(content))
            select content.markdown
    });

var posts = await jinagaClient.Query(postsInSite, site);

posts

index value 0 { Post = Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM }, Title = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], Content = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } Post Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM } site Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM } creator Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:05Z author Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:09Z Title [ Interesting Facts ] (values) [ Interesting Facts ] Content [ Let me tell you about this thing that I love. ] (values) [ Let me tell you about this thing that I love. ]

Create a view model for the child screen in the same way you did for the parent.

In [34]:
var postsViewModel = posts.Select(p => new
{
    Title = p.Title.FirstOrDefault() ?? "New post",
    Content = p.Content.FirstOrDefault() ?? ""
});

postsViewModel

index value 0 { Title = Interesting Facts, Content = Let me tell you about this thing that I love. } Title Interesting Facts Content Let me tell you about this thing that I love.

## Tree View

If your application calls for it, you can display the hierarchy in a single view.
You probably wouldn't do this for a blog application, but I'll demonstrate how this is done.

Define a specification that starts at the top level, in this case the creator `User` of the sites.
Then extend that specification with child specifications in the `select` clause.
Continue for as many levels as you would like to display in the tree.

In [35]:
var sitesAndPostsByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    where !facts.Any<SiteDeleted>(
        sd => sd.site == site &&
        !facts.Any<SiteRestored>(sr => sr.deleted == sd))
    select new
    {
        Site = site,
        Names =
            from name in facts.OfType<SiteName>()
            where name.site == site
            where !facts.Any<SiteName>(next => next.prior.Contains(name))
            select name.value,
        Domains =
            from domain in facts.OfType<SiteDomain>()
            where domain.site == site
            where !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
            select domain.value,
        Posts =
            from post in facts.OfType<Post>()
            where post.site == site
            where !facts.Any<PostDeleted>(pd => pd.post == post &&
                !facts.Any<PostRestored>(pr => pr.deleted == pd))
            select new
            {
                Post = post,
                Title =
                    from title in facts.OfType<PostTitle>()
                    where title.post == post
                    where !facts.Any<PostTitle>(next => next.prior.Contains(title))
                    select title.value
            }
    });

var sitesAndPosts = await jinagaClient.Query(sitesAndPostsByUser, user);

sitesAndPosts

index value 0 { Site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, Names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], Domains = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], Posts = System.Linq.Enumerable+<OfTypeIterator>d__66`1[<>f__AnonymousType1#34`2[S... Site Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM } creator Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:05Z Names [ My Journal ] (values) [ My Journal ] Domains [ example.com ] (values) [ example.com ] Posts [ { Post = Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM }, Title = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } ] (values) index value 0 { Post = Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM }, Title = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } Post Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM } site Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM } creator Jinaga.User createdAt 5/18/2024 5:54:05 PM author Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:09Z Title [ Interesting Facts ] (values) [ Interesting Facts ]

Create a hierarchical view model by selecting the child view models from the child objects.

In [36]:

var sitesAndPostsViewModel = sitesAndPosts.Select(s => new
{
    Name = s.Names.FirstOrDefault() ?? "New site",
    Domain = s.Domains.FirstOrDefault() ?? "",
    Posts = s.Posts.Select(p => new
    {
        Title = p.Title.FirstOrDefault() ?? "New post"
    })
});

sitesAndPostsViewModel

index value 0 { Name = My Journal, Domain = example.com, Posts = System.Linq.Enumerable+<OfTypeIterator>d__66`1[<>f__AnonymousType1#34`2[Submission#29+Post,System.Linq.IQueryable`1[System.String]]].Select(p => new <>f__AnonymousType1#35`1(Title = (p.Title.FirstOrDefault() ?? "New post"))) } Name My Journal Domain example.com Posts [ { Title = Interesting Facts } ] (values) index value 0 { Title = Interesting Facts } Title Interesting Facts

The hierarchical pattern described here works for models that have different kinds of objects at each level.
If you have a hierarchy that have the same kinds of objects at various levels -- such as folders that contain folders, or employees who manage other employees -- then the pattern is different.
We will discuss that elsewhere.

## Workflow

So far we have modeled entities with facts.
We used facts to create, delete, and restore the entities.
We also used facts to change the values of their mutable properties.
We were able to create child entities and define an application hierarchy.

But entities are not the only things that facts can represent.
They can also represent steps in a workflow.
Indeed, this is where facts truly gain their expressive power.

The creator of a blog can write posts in private.
Once they are satisfied, they can publish the post.
They will record this decision to publish using a new fact.
This fact will capture the specific title and content that the author chose to publish.
Finally, it will refer back to prior published versions so that they can be replaced.

In [37]:
[FactType("Blog.Post.Publish")]
public record PostPublish(Post post, PostTitle title, PostContent content, PostPublish[] prior) { }

Renderer.RenderTypes(typeof(PostPublish))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Post.Publish -->
 
 Blog.Post.Publish 
 
 Blog.Post.Publish 
 
<!-- Blog.Post.Publish->Blog.Post.Publish -->
 
 Blog.Post.Publish->Blog.Post.Publish 
 
 
 prior 
 
<!-- Blog.Post -->
 
 Blog.Post 
 
 Blog.Post 
 
<!-- Blog.Post.Publish->Blog.Post -->
 
 Blog.Post.Publish->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Title -->
 
 Blog.Post.Title 
 
 Blog.Post.Title 
 
<!-- Blog.Post.Publish->Blog.Post.Title -->
 
 Blog.Post.Publish->Blog.Post.Title 
 
 
 title 
 
<!-- Blog.Post.Content -->
 
 Blog.Post.Content 
 
 Blog.Post.Content 
 
<!-- Blog.Post.Publish->Blog.Post.Content -->
 
 Blog.Post.Publish->Blog.Post.Content 
 
 
 content 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Post->Blog.Site -->
 
 Blog.Post->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Post->Jinaga.User -->
 
 Blog.Post->Jinaga.User 
 
 
 author 
 
<!-- Blog.Post.Title->Blog.Post -->
 
 Blog.Post.Title->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Title->Blog.Post.Title -->
 
 Blog.Post.Title->Blog.Post.Title 
 
 
 prior 
 
<!-- Blog.Post.Content->Blog.Post -->
 
 Blog.Post.Content->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Content->Blog.Post.Content -->
 
 Blog.Post.Content->Blog.Post.Content 
 
 
 prior 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

When the user makes the decision to publish the post, we capture that as a fact.

In [38]:
var post1Publish0 = await jinagaClient.Fact(new PostPublish(post1, postTitle1, postContent1, []));

jinagaClient.RenderFacts(post1Publish0)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 Blog.Post 
 
 createdAt 
 
 2024-05-18T17:54:09.... 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 author 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A== -->
 
 aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A== 
 
 Blog.Post.Title 
 
 value 
 
 Interesting Facts 
 
<!-- aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 
 post 
 
<!-- VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== -->
 
 VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== 
 
 Blog.Post.Content 
 
 markdown 
 
 Let me tell you abou... 
 
<!-- VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 
 post 
 
<!-- CFuc4+r+zy1knxFd9yAyuILW4IyYcIp8YT+3N/93D9d4KrHaxHYfbEdBQIs1ZeVaTwPikmGFL02nrB4sy1oCdA== -->
 
 CFuc4+r+zy1knxFd9yAyuILW4IyYcIp8YT+3N/93D9d4KrHaxHYfbEdBQIs1ZeVaTwPikmGFL02nrB4sy1oCdA== 
 
 Blog.Post.Publish 
 
 
<!-- CFuc4+r+zy1knxFd9yAyuILW4IyYcIp8YT+3N/93D9d4KrHaxHYfbEdBQIs1ZeVaTwPikmGFL02nrB4sy1oCdA==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 CFuc4+r+zy1knxFd9yAyuILW4IyYcIp8YT+3N/93D9d4KrHaxHYfbEdBQIs1ZeVaTwPikmGFL02nrB4sy1oCdA==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 

To render the site, create a specification that displays only the published posts.

Here you have to be careful.
You might at first want to write a specification that selects predecessors and properties of predecessors like this:

In [39]:
var publishedPostsInSiteIncorrect = Given<Site>.Match((site, facts) =>
    from publish in facts.OfType<PostPublish>()
    where publish.post.site == site
    where !facts.Any<PostPublish>(next => next.prior.Contains(publish))
    select new
    {
        // Get the post predecessor from the publish fact
        Post = publish.post,
        // Get the value of the title predecessor
        Title = publish.title.value,
        // Get the value of the content predecessor
        Content = publish.content.markdown
    });

Error: Jinaga.SpecificationException: Cannot select post directly. Give the fact a label first.
   at Jinaga.Repository.SpecificationProcessor.ProcessProjection(Expression expression, SymbolTable symbolTable)
   at Jinaga.Repository.SpecificationProcessor.<>c__DisplayClass7_0.<ProcessProjection>b__2(Expression arg)
   at System.Linq.Enumerable.SelectIListIterator`2.MoveNext()
   at System.Linq.Enumerable.ZipIterator[TFirst,TSecond,TResult](IEnumerable`1 first, IEnumerable`1 second, Func`3 resultSelector)+MoveNext()
   at System.Collections.Immutable.ImmutableDictionary`2.AddRange(IEnumerable`1 items, MutationInput origin, KeyCollisionBehavior collisionBehavior)
   at System.Collections.Immutable.ImmutableDictionary`2.AddRange(IEnumerable`1 pairs, Boolean avoidToHashMap)
   at Jinaga.Repository.SpecificationProcessor.ProcessProjection(Expression expression, SymbolTable symbolTable)
   at Jinaga.Repository.SpecificationProcessor.ProcessSource(Expression expression, SymbolTable symbolTabl

That is an invalid specification.
The problem is that we are trying to select a predecessor directly in the projection.
We can't know for sure that just one predecessor exists.
The model may have changed over time.
That predecessor might have been added, or it might have changed from an array to a single value.
Jinaga does not make any assumptions about the evolution of the model.

Instead, you have to explicitly give each predecessor a label.
Then, you can select fields of those predecessors in the projection.

In [40]:
var publishedPostsInSite = Given<Site>.Match((site, facts) =>
    from publish in facts.OfType<PostPublish>()
    where publish.post.site == site
    where !facts.Any<PostPublish>(next => next.prior.Contains(publish))
    from post in facts.OfType<Post>()
    where post == publish.post
    from title in facts.OfType<PostTitle>()
    where title == publish.title
    from content in facts.OfType<PostContent>()
    where content == publish.content
    select new
    {
        Post = post,
        Title = title.value,
        Content = content.markdown
    });

var publishedPosts = await jinagaClient.Query(publishedPostsInSite, site);

publishedPosts

index value 0 { Post = Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM }, Title = Interesting Facts, Content = Let me tell you about this thing that I love. } Post Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM } site Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM } creator Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:05Z author Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:09Z Title Interesting Facts Content Let me tell you about this thing that I love.

If the publish fact does not have a title or content predecessor, then this specification will not return the result.
Furthermore, if it has multiple title or content predecessors -- for example if the model previously captured arrays instead of single values -- then it would return one result per combination.
This might not be your desired result.

An alternate specification might explicitly say that it wants one result per published post, no matter how many title or content facts are present.
It then gets the list of titles and contents within the projection.
If multiples exist, then they will be listed in these collections.

In [41]:
var publishedPostsInSiteAlternate = Given<Site>.Match((site, facts) =>
    from publish in facts.OfType<PostPublish>()
    where publish.post.site == site
    where !facts.Any<PostPublish>(next => next.prior.Contains(publish))
    from post in facts.OfType<Post>()
    where post == publish.post
    select new
    {
        Post = post,
        Titles =
            from title in facts.OfType<PostTitle>()
            where title == publish.title
            select title.value,
        Contents =
            from content in facts.OfType<PostContent>()
            where content == publish.content
            select content.markdown
    });

var publishedPostsAlternate = await jinagaClient.Query(publishedPostsInSiteAlternate, site);

publishedPostsAlternate

index value 0 { Post = Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM }, Titles = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], Contents = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } Post Post { site = Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM }, author = Jinaga.User, createdAt = 5/18/2024 5:54:09 PM } site Site { creator = Jinaga.User, createdAt = 5/18/2024 5:54:05 PM } creator Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:05Z author Jinaga.User publicKey --- FAKE USER --- createdAt 2024-05-18 17:54:09Z Titles [ Interesting Facts ] (values) [ Interesting Facts ] Contents [ Let me tell you about this thing that I love. ] (values) [ Let me tell you about this thing that I love. ]

Jinaga does not assume that you want one form vs the other.
You must explicitly label the facts that you want to be present in the projection.
From there, you can define nested projections to pick out related facts.

## Captured Versions

Because the `Publish` fact captures the title and content that the author wishes to display on the site, the author can make changes without affecting what appears on the site.

In [42]:
var editedPostContent1 = await jinagaClient.Fact(new PostContent(post1, "Let me tell you some more.", [postContent1]));

jinagaClient.RenderFacts(editedPostContent1)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 Blog.Post 
 
 createdAt 
 
 2024-05-18T17:54:09.... 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 author 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== -->
 
 VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== 
 
 Blog.Post.Content 
 
 markdown 
 
 Let me tell you abou... 
 
<!-- VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 
 post 
 
<!-- grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ== -->
 
 grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ== 
 
 Blog.Post.Content 
 
 markdown 
 
 Let me tell you some... 
 
 
<!-- grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 
 post 
 
<!-- grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ==->VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== -->
 
 grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ==->VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== 
 
 
 prior

The author can see the edits they are making.
Their view model selects the current content of a post.

In [43]:
posts = await jinagaClient.Query(postsInSite, site);

postsViewModel = posts.Select(p => new
{
    Title = p.Title.FirstOrDefault() ?? "New post",
    Content = p.Content.FirstOrDefault() ?? ""
});

postsViewModel

index value 0 { Title = Interesting Facts, Content = Let me tell you some more. } Title Interesting Facts Content Let me tell you some more.

However, visitors to the site will still see the published version of the post.
Their view model is based on the `Publish` fact.

In [44]:
publishedPosts = await jinagaClient.Query(publishedPostsInSite, site);

var publishedPostsViewModel = publishedPosts.Select(p => new
{
    Title = p.Title,
    Content = p.Content
});

publishedPostsViewModel

index value 0 { Title = Interesting Facts, Content = Let me tell you about this thing that I love. } Title Interesting Facts Content Let me tell you about this thing that I love.

When the author is ready, they can publish the updated version.

In [45]:
var post1Publish1 = await jinagaClient.Fact(new PostPublish(post1, postTitle1, editedPostContent1, [post1Publish0]));

jinagaClient.RenderFacts(post1Publish1)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T17:54:05.... 
 
<!-- E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 Blog.Post 
 
 createdAt 
 
 2024-05-18T17:54:09.... 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 author 
 
<!-- 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== -->
 
 3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ==->E9Yqy7P2Os7+KzyjERqubfVs+l4HPSiYpjU1kmkWNNbkUETEQ0Mz/OPSn6Y9LtND5jboFWEiLE7m2R/qUrkGgA== 
 
 
 site 
 
<!-- aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A== -->
 
 aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A== 
 
 Blog.Post.Title 
 
 value 
 
 Interesting Facts 
 
<!-- aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 aV2fb+3w5Bp+drZkecDG+J1F3cH2E5npUEwyVA7bQldu7QB2TYbM91ytSP0INx+kDJ0wBhMKNuY015IKvT0N3A==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 
 post 
 
<!-- VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== -->
 
 VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg== 
 
 Blog.Post.Content 
 
 markdown 
 
 Let me tell you abou... 
 
<!-- VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 VkZXyLeUOG4ieLSo99kwEGE5J7p4XUOBvxtkDHgcY+wbqUsDk0cO1NyHvqN6X816AI6cpcyBEOzit4gP13/SOg==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== 
 
 
 post 
 
<!-- grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ== -->
 
 grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ== 
 
 Blog.Post.Content 
 
 markdown 
 
 Let me tell you some... 
 
<!-- grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+g13PacHaars9G8YhOce86j5oiSrUwGRghQ== -->
 
 grunMfMQpcfIhdgee+4PqQlOlifk4B+JZcFSlValwnQDNqeRzyyt+UOXM+qjlo4x7lTUUbD6Flq6pjj8azejZQ==->3lOU4D149O36O35x/SROJpLkOhfDj0TAC27vzSKUR0Lr7t+bOPK+

Now visitors to the site will see the updated content.

In [46]:
publishedPosts = await jinagaClient.Query(publishedPostsInSite, site);

publishedPostsViewModel = publishedPosts.Select(p => new
{
    Title = p.Title,
    Content = p.Content
});

publishedPostsViewModel

index value 0 { Title = Interesting Facts, Content = Let me tell you some more. } Title Interesting Facts Content Let me tell you some more.

## Additional Workflow Steps

Workflow often includes more than a single step.
To represent additional steps in a workflow, define more fact types.
Each step refers back to the previous step in the workflow.

After a post is published, it can be rescinded.
The author might want to take it down for a time, or permanently remove it.
Represent this with another fact type.

In [47]:
[FactType("Blog.Post.Rescind")]
public record PostRescind(PostPublish publish, DateTime rescindedAt) { }

Renderer.RenderTypes(typeof(PostRescind))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Post.Rescind -->
 
 Blog.Post.Rescind 
 
 Blog.Post.Rescind 
 
<!-- Blog.Post.Publish -->
 
 Blog.Post.Publish 
 
 Blog.Post.Publish 
 
<!-- Blog.Post.Rescind->Blog.Post.Publish -->
 
 Blog.Post.Rescind->Blog.Post.Publish 
 
 
 publish 
 
<!-- Blog.Post.Publish->Blog.Post.Publish -->
 
 Blog.Post.Publish->Blog.Post.Publish 
 
 
 prior 
 
<!-- Blog.Post -->
 
 Blog.Post 
 
 Blog.Post 
 
<!-- Blog.Post.Publish->Blog.Post -->
 
 Blog.Post.Publish->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Title -->
 
 Blog.Post.Title 
 
 Blog.Post.Title 
 
<!-- Blog.Post.Publish->Blog.Post.Title -->
 
 Blog.Post.Publish->Blog.Post.Title 
 
 
 title 
 
<!-- Blog.Post.Content -->
 
 Blog.Post.Content 
 
 Blog.Post.Content 
 
<!-- Blog.Post.Publish->Blog.Post.Content -->
 
 Blog.Post.Publish->Blog.Post.Content 
 
 
 content 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Post->Blog.Site -->
 
 Blog.Post->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Post->Jinaga.User -->
 
 Blog.Post->Jinaga.User 
 
 
 author 
 
<!-- Blog.Post.Title->Blog.Post -->
 
 Blog.Post.Title->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Title->Blog.Post.Title -->
 
 Blog.Post.Title->Blog.Post.Title 
 
 
 prior 
 
<!-- Blog.Post.Content->Blog.Post -->
 
 Blog.Post.Content->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Content->Blog.Post.Content -->
 
 Blog.Post.Content->Blog.Post.Content 
 
 
 prior 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

Modify the specifications that should take these workflow steps into account.
For example, we don't want to display rescinded posts on the site.

In [48]:
publishedPostsInSite = Given<Site>.Match((site, facts) =>
    from publish in facts.OfType<PostPublish>()
    where publish.post.site == site
    where !facts.Any<PostPublish>(next => next.prior.Contains(publish))
    // Filter out the rescinded posts
    where !facts.Any<PostRescind>(pr => pr.publish == publish)
    from post in facts.OfType<Post>()
    where post == publish.post
    from title in facts.OfType<PostTitle>()
    where title == publish.title
    from content in facts.OfType<PostContent>()
    where content == publish.content
    select new
    {
        Post = post,
        Title = title.value,
        Content = content.markdown
    });

Now if the author rescinds their publication, the post will no longer appear on the site.

In [49]:
var rescindPost1Publish1 = await jinagaClient.Fact(new PostRescind(post1Publish1, DateTime.UtcNow));

publishedPosts = await jinagaClient.Query(publishedPostsInSite, site);

publishedPostsViewModel = publishedPosts.Select(p => new
{
    Title = p.Title,
    Content = p.Content
});

publishedPostsViewModel

(empty)

The next step in the workflow might be to re-publish a rescinded post.
As you might expect, we represent that with another fact.

In [50]:
[FactType("Blog.Post.Republish")]
public record PostRepublish(PostRescind rescind) { }

Renderer.RenderTypes(typeof(PostRepublish))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Post.Republish -->
 
 Blog.Post.Republish 
 
 Blog.Post.Republish 
 
<!-- Blog.Post.Rescind -->
 
 Blog.Post.Rescind 
 
 Blog.Post.Rescind 
 
<!-- Blog.Post.Republish->Blog.Post.Rescind -->
 
 Blog.Post.Republish->Blog.Post.Rescind 
 
 
 rescind 
 
<!-- Blog.Post.Publish -->
 
 Blog.Post.Publish 
 
 Blog.Post.Publish 
 
<!-- Blog.Post.Rescind->Blog.Post.Publish -->
 
 Blog.Post.Rescind->Blog.Post.Publish 
 
 
 publish 
 
<!-- Blog.Post.Publish->Blog.Post.Publish -->
 
 Blog.Post.Publish->Blog.Post.Publish 
 
 
 prior 
 
<!-- Blog.Post -->
 
 Blog.Post 
 
 Blog.Post 
 
<!-- Blog.Post.Publish->Blog.Post -->
 
 Blog.Post.Publish->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Title -->
 
 Blog.Post.Title 
 
 Blog.Post.Title 
 
<!-- Blog.Post.Publish->Blog.Post.Title -->
 
 Blog.Post.Publish->Blog.Post.Title 
 
 
 title 
 
<!-- Blog.Post.Content -->
 
 Blog.Post.Content 
 
 Blog.Post.Content 
 
<!-- Blog.Post.Publish->Blog.Post.Content -->
 
 Blog.Post.Publish->Blog.Post.Content 
 
 
 content 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Post->Blog.Site -->
 
 Blog.Post->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Post->Jinaga.User -->
 
 Blog.Post->Jinaga.User 
 
 
 author 
 
<!-- Blog.Post.Title->Blog.Post -->
 
 Blog.Post.Title->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Title->Blog.Post.Title -->
 
 Blog.Post.Title->Blog.Post.Title 
 
 
 prior 
 
<!-- Blog.Post.Content->Blog.Post -->
 
 Blog.Post.Content->Blog.Post 
 
 
 post 
 
<!-- Blog.Post.Content->Blog.Post.Content -->
 
 Blog.Post.Content->Blog.Post.Content 
 
 
 prior 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

Then we update the specification to ignore a rescind fact if it has been republished.

In [51]:
publishedPostsInSite = Given<Site>.Match((site, facts) =>
    from publish in facts.OfType<PostPublish>()
    where publish.post.site == site
    where !facts.Any<PostPublish>(next => next.prior.Contains(publish))
    where !facts.Any<PostRescind>(pr => pr.publish == publish &&
        // Include only the rescinds that have not been republished
        !facts.Any<PostRepublish>(rp => rp.rescind == pr))
    from post in facts.OfType<Post>()
    where post == publish.post
    from title in facts.OfType<PostTitle>()
    where title == publish.title
    from content in facts.OfType<PostContent>()
    where content == publish.content
    select new
    {
        Post = post,
        Title = title.value,
        Content = content.markdown
    });

And with this, the author can re-publish their rescinded post.

In [52]:
var republishPost1Rescind = await jinagaClient.Fact(new PostRepublish(rescindPost1Publish1));

publishedPosts = await jinagaClient.Query(publishedPostsInSite, site);

publishedPostsViewModel = publishedPosts.Select(p => new
{
    Title = p.Title,
    Content = p.Content
});

publishedPostsViewModel

index value 0 { Title = Interesting Facts, Content = Let me tell you some more. } Title Interesting Facts Content Let me tell you some more.

The decision of which workflow facts have timestamps is very deliberate.
It must be considered carefully.
A timestamp is added to a workflow fact only when it is needed to differentiate it from other facts.

Let's first consider the `PostPublish` fact.
It has no timestamp.
The reason is that it contains a `prior` array that places it in history relative to other `PostPublish` facts.
This is enough to distinguish it from other publication steps.

Then let's look at the `PostRescind` fact.
This one has a timestamp.
Without this differentiator, there could be only one `PostRescind` for a given `PostPublish`.
An author could rescind a post and then re-publish it.
After that, they could not express the desire to rescind it again.
That second rescind would be indistinguishable from the first.

Finally, let's look at `PostRepublish`.
This one does not have a timestamp.
It does not need one because it is the last step in the workflow.
If an author wishes to rescind the publication again, they could create another `PostRescind` fact.
There is no need to further qualify a re-publish fact, and therefore no need to differentiate it from any other re-publication of the same rescinded post.